In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Embedding, Concatenate
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder


dataframe = pd.read_csv('data/processed.csv')

dataframe = dataframe.sample(frac=0.1, random_state=1)

print("Shape of dataframe: ", dataframe.shape)
dataframe.head()

Shape of dataframe:  (103113, 8)


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age
887470,4556,8,Abduction,Robin Cook,2000,Berkley Publishing Group,"south berwick, maine, usa",34
56511,234623,0,Nancy Kerrigan: In My Own Words,Nancy Kerrigan,1996,Disney Pr (Jp),"scarborough, maine, usa",34
238538,190925,0,Bad Girl : A Novel,MICHELE JAFFE,2003,Ballantine Books,"hobe sound, florida, usa",51
610781,56661,0,The Joy Luck Club,Amy Tan,1994,Prentice Hall (K-12),"green, ohio, usa",34
179676,56856,0,The Pleasure of My Company: A Novel,Steve Martin,2003,Hyperion,"escondido, california, usa",23


In [7]:
nUsers = dataframe['User-ID'].nunique()
nBooks = dataframe['Book-Title'].nunique()

In [8]:
userInput = Input(shape=(1,))
userEmbedding = Embedding(input_dim=nUsers, output_dim=32)(userInput)
userVector = Flatten()(userEmbedding)
userOutput = Dense(1)(userVector)

bookInput = Input(shape=(1,))
bookEmbedding = Embedding(input_dim=nBooks, output_dim=32)(bookInput)
bookVector = Flatten()(bookEmbedding)
bookOutput = Dense(1)(bookVector)

concatenate = Concatenate()([userVector, bookVector])

dense = Dense(128, activation='relu')(concatenate)
output = Dense(1)(dense)

In [9]:
model = Model([userInput, bookInput], output)
model.compile(loss='mean_squared_error', optimizer='adam')

early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

In [10]:
userEncoder = LabelEncoder()
userIDs = userEncoder.fit_transform(dataframe['User-ID'].values)

bookEncoder = LabelEncoder()
bookIDs = bookEncoder.fit_transform(dataframe['Book-Title'].values)

model.fit([userIDs, bookIDs], bookIDs, epochs=5, verbose=1)

Epoch 1/5
3223/3223 [==============================] - 20s 6ms/step - loss: 790768960.0000
Epoch 2/5
3223/3223 [==============================] - 20s 6ms/step - loss: 301254880.0000
Epoch 3/5
3223/3223 [==============================] - 19s 6ms/step - loss: 132368184.0000
Epoch 4/5
3223/3223 [==============================] - 19s 6ms/step - loss: 52525640.0000
Epoch 5/5
3223/3223 [==============================] - 19s 6ms/step - loss: 20789404.0000


In [11]:
def recommend(userID, n):
    books = dataframe.copy()
    books['User-ID'] = userID
    books['User-ID'] = userEncoder.transform(books['User-ID'])
    books['Book-Title'] = bookEncoder.transform(books['Book-Title'])

    predictions = model.predict([books['User-ID'], books['Book-Title']])
    books['prediction'] = predictions
    books = books.drop_duplicates(subset=['Book-Title'])
    books = books.sort_values(by='prediction', ascending=False)
    bookTitles = books['Book-Title'].head(n).values

    print("\nRecommendations for user: ", userID)

    for i in range(n):
        print(bookEncoder.inverse_transform([bookTitles[i]]))

recommend(276747, 10)

3223/3223 [==============================] - 1s 392us/step

Recommendations for user:  276747
['Yonder : A Place in Montana']
['Words I Wish I Wrote: A Collection of Writing That Inspired My Ideas']
['Whispers at Midnight']
["ZACHARY'S WINGS: A Novel"]
['Zigzag Street ;']
['\\Girls\' Night Out\\"/\\"Boys\' Night In\\""']
["Winter's Tale"]
['Wild Life']
['Wlt: A Radio Romance']
['When the Emperor Was Divine']
